In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Your task is to modify `KNNClassifier` class from your practice in class. The `KNNClassifier` class with empty methods is provided below. Please, modify it to do all tasks.

In [39]:
from collections import Counter

from sklearn.neighbors import KDTree

class KNNClassifier(object):
    """
    Omg, this code is horrible...
    """
    
    def __init__(self, max_dist=1., use_kd_tree=False, use_weights=False):
        """
        This is a constructor of the class. 
        Here you can define parameters (max_dist) of the class and 
        attributes, that are visible within all methods of the class.
        
        Parameters
        ----------
        max_dist : float
            Maximum distance between an object and its neighbors.
        """
        
        # Make this parameter visible in all methods of the class
        self.max_dist = max_dist
        self.use_kd_tree = use_kd_tree
        self.use_weights = use_weights
        
        self.kd_tree = None
        self.x_train = None
        self.y_train = None
    
    def fit(self, X, y):
        """
        This method trains the KNN classifier. 
        Actualy, the KNN classifier has no training procedure.
        It just remembers data (X, y) that will be used for predictions.
        
        Parameters
        ----------
        X : numpy.array, shape = (n_objects, n_features)
            Matrix of objects that are described by their input features.
        y : numpy.array, shape = (n_objects)
            1D array with the object labels. 
            For the classification labels are integers in {0, 1, 2, ...}.
        """
        
        ### Your code here
        
        self.x_train = X
        self.y_train = y
        if self.use_kd_tree:
            self.kd_tree = KDTree(self.x_train, leaf_size=30)
    
    def calculate_distances(self, X, one_x):
        """
        This method calculates distances between one object and all other objects.
        
        Parameters
        ----------
        X : numpy.array, shape = (n_objects, n_features)
            Matrix of objects that are described by their input features.
        one_x : numpy.array, shape = (n_features)
        """
        
        dists = np.sqrt( np.sum( (X - one_x)**2, axis=1 ) )
        return dists
    
    def _calculate_weight_for_point(self, x, x_pred):
        return 1 / ((x[0] - x_pred[0]) ** 2 + (x[1] - x_pred[1]) ** 2) ** 0.5
    
    def _is_in_radius(self, x, check):
        """
        This method checks, whether check is within x radius or not
        x : numpy.array, shape = (2)
        check : numpy.array, shape = (2)
        """
        return (x[0] - check[0]) ** 2 + (x[1] - check[1]) ** 2 < self.max_dist ** 2
    
    def _get_closest_point(self, x):
        """
        This method gets a point as an input and return index of a closest point to it.
        x : numpy.array, shape = (2)
        returns:
        index: int
        """
        distances = self.calculate_distances(self.x_train, x)
        return np.argmin(distances)
        
    
    def _get_closest_neighbours(self, x):
        """
        This method gets a point as an input and returns indeces for features and labels of closest dots.
        x : numpy.array, shape = (2)
        """
        indxs = []
        for i in range(len(self.x_train)):
            if self._is_in_radius(x, self.x_train[i]):
                indxs.append(i)
        # This means that there is no point in our radius and we need to find closest.
        if len(indxs) == 0:
            indxs.append(self._get_closest_point(x))
        return indxs
    
    def _predict_for_x(self, x):
        """
        This method gets an x and returns predicted label y
        """
        indxs = self._get_closest_neighbours(x)
        labels = Counter()
        # here we decide, which type of weights to use
        if self.use_weights:
            # Here we will have array of tuples, each tuple will be (weight, label)
            weights = [(self._calculate_weight_for_point(x, self.x_train[index]), self.y_train[index]) for index in indxs]
            for weight in weights:
                labels[weight[1]] += weight[0]
        else:
            labels.update(Counter([self.y_train[i] for i in indxs]))
        # This function returns list of tuples, that's why we need such magic in order to get most_common class
        return labels.most_common(1)[0][0]
    
    def _predict_for_x_with_kd_tree(self, X):
        """
        This method just use kd tree instead my own shitty implementation
        """
        y_pred = []
        indxs = self.kd_tree.query_radius(X, self.max_dist)
        k_means_indxs = self.kd_tree.query(X)
        for i, indx in enumerate(indxs):
            # That means that there is some points in the radius
            if len(indx) > 0:
                labels = Counter()
                if self.use_weights:
                    weights = [(self._calculate_weight_for_point(X[i], self.x_train[j]), self.y_train[j]) for j in indx]
                    for weight in weights:
                        labels[weight[1]] += weight[0]
                else:
                    labels.update(Counter([self.y_train[j] for j in indx]))
                y_pred.append(labels.most_common(1)[0][0])
            else:
                label = self.y_train[k_means_indxs[i][0]]
                y_pred.append(label)
        return y_pred
        
        
    def predict(self, X):
        """
        This methods performs labels prediction for new objects.
        
        Parameters
        ----------
        X : numpy.array, shape = (n_objects, n_features)
            Matrix of objects that are described by their input features.
            
        Returns
        -------
        y_predicted : numpy.array, shape = (n_objects)
            1D array with predicted labels. 
            For the classification labels are integers in {0, 1, 2, ...}.
        """
        
        # Create an empty list for predicted labels
        y_predicted = []
        if self.use_kd_tree:
            y_predicted = self._predict_for_x_with_kd_tree(X)
        else:
        ### Replace this line with your code:
            for x in X:
                y_predicted.append(self._predict_for_x(x))
        
        ### The end of your code
            
        return np.array(y_predicted) # return numpy.array
    
    
    def predict_proba_for_point(self, x):
        """
        This methods performs prediction of probabilites of each class for new object.
        """
        kd_indxs = self.kd_tree.query_radius(np.array([x]), self.max_dist)
        neighbours = self._get_closest_neighbours(x)
        weights = {i:0 for i in range(max(self.y_train) + 1)}
        if self.use_kd_tree:
            # Since we won't to give full proba for point, we need to take care of classes, which are not in radius also.

            if self.use_weights:
                w = [(self._calculate_weight_for_point(x, self.x_train[i]), self.y_train[i]) for i in kd_indxs[0]]
                for item in w:
                    weights[item[1]] += item[0]
            else:
                for i in kd_indxs[0]:
                    weights[self.y_train[i]] += 1
        else:
            if self.use_weights:
                w = [(self._calculate_weight_for_point(x, self.x_train[index]), self.y_train[index]) for index in neighbours]
                for item in w:
                    item[weight[1]] += item[0]
            else:
                for i in neighbours:
                    weights[self.y_train[i]] += 1
        proba = [weights[item] / sum(weights.values()) for item in sorted(weights.keys())]
        return proba
                

                    
                
    
    def predict_proba(self, X):
        """
        This methods performs prediction of probabilities of each class for new objects.
        
        Parameters
        ----------
        X : numpy.array, shape = (n_objects, n_features)
            Matrix of objects that are described by their input features.
            
        Returns
        -------
        y_predicted_proba : numpy.array, shape = (n_objects, n_classes)
            2D array with predicted probabilities of each class. 
            Example:
                y_predicted_proba = [[0.1, 0.9],
                                     [0.8, 0.2], 
                                     [0.0, 1.0], 
                                     ...]
        """
        
        # Create an empty list for predictions
        y_predicted_proba = []
        
        ### Replace these lines with your code:
        for x in X:
            y_predicted_proba.append(self.predict_proba_for_point(x))
        ### The end of your code
            
        return np.array(y_predicted_proba) # return numpy.array

### Task 1 (1 point) <br/>
Create a matrix of object features `X` and vector of labels `y` for N=1000 objects using `sklearn.datasets.make_moons()` function from scikit-learn library. Also, set up random state in the function `random_state=42` and `noise=0.2`. To open the function description use `Shift` + `Tab` .

In [41]:
### Your code here
from sklearn.datasets import make_moons
N = 1000
X, y = make_moons(n_samples=N, noise=0.2, random_state=42)


### Check your solution
ans = np.array([[-0.112,  0.52 ],
                [ 1.143, -0.343]])
assert np.array_equal(np.round(X[:2], 3), ans), ('Check your solution.')

### Task 2 (1 point) <br/>

Split the sample into train and test samples using `sklearn.model_selection.train_test_split()` function from scikit-learn library. Use `random_state = 42` and `test_size = 0.5`.

In [42]:
### Your code here
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


### Check your solution
ans = np.array([[ 0.77 , -0.289],
                [ 0.239,  1.041]])
assert np.array_equal(np.round(X_train[:2], 3), ans), ('Check your solution.')
max(y_train)

1

### Task 3 (2 points) <br/>

Modify class `KNNClassifier` above and implement `predict()` method that uses **max_dist** parameter to select neighbors like it's shown in the second figure (radius search). If there is no any object within **max_dist**, make decision based on the closest neighbor.

<img src="img/knn2.png" width="600">

In [43]:
# Create a class object
knn = KNNClassifier(max_dist=0.5)

# Train the classifier
knn.fit(X_train, y_train)

# Make prediction using the trained classifier
%time y_test_predict = knn.predict(X_test) # measure time for prediction

# Import accuracy_score function
from sklearn.metrics import accuracy_score
# Calculate accuracy for the test sample
accuracy_test = accuracy_score(y_test, y_test_predict)
print("Test accuracy of KNN classifier: ", accuracy_test)



### Check your solution
assert accuracy_test == 0.964, ('Check your solution.')

CPU times: user 696 ms, sys: 7.97 ms, total: 704 ms
Wall time: 717 ms
Test accuracy of KNN classifier:  0.964


### Task 4 (2 points) <br/>

There are an algorithm [kd-tree](https://en.wikipedia.org/wiki/K-d_tree) that helps to find neighbors faster. Using [scipy.spatial.cKDTree](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.cKDTree.html#scipy.spatial.cKDTree) function modify you classifier to speed up **predict** method. Use `leafsize=30` in `KDTree`. Similar to `max_dist` option, add option `use_kd_tree = True/False` to your classifier.

In [44]:
# Create a class object
knn = KNNClassifier(max_dist=0.5, use_kd_tree=True)

# Train the classifier
knn.fit(X_train, y_train)

# Make prediction using the trained classifier
%time y_test_predict = knn.predict(X_test) # measure time for prediction

# Import accuracy_score function
from sklearn.metrics import accuracy_score
# Calculate accuracy for the test sample
accuracy_test = accuracy_score(y_test, y_test_predict)
print("Test accuracy of KNN classifier: ", accuracy_test)

### Check your solution
assert accuracy_test == 0.964, ('Check your solution.')

CPU times: user 49.3 ms, sys: 10.3 ms, total: 59.5 ms
Wall time: 73.2 ms
Test accuracy of KNN classifier:  0.964


### Task 5 (3 points) <br/>

Now modify the **predict** method to provide prediction with neighbors weights.

<img src="img/wv1.png">

<img src="img/wv2.png">

We propose you to use the following weights:

$$
w_{i} = \frac{1}{\rho(x, x_{i})}
$$

Similar to `max_dist` option, add option `use_weights = True/False` to your classifier.

In [45]:
# Create a class object
knn = KNNClassifier(max_dist=0.5, use_kd_tree=True, use_weights=True)

# Train the classifier
knn.fit(X_train, y_train)

# Make prediction using the trained classifier
%time y_test_predict = knn.predict(X_test) # measure time for prediction

# Import accuracy_score function
from sklearn.metrics import accuracy_score
# Calculate accuracy for the test sample
accuracy_test = accuracy_score(y_test, y_test_predict)
print("Test accuracy of KNN classifier: ", accuracy_test)



### Check your solution
assert accuracy_test == 0.968, ('Check your solution.')

CPU times: user 175 ms, sys: 4.25 ms, total: 180 ms
Wall time: 189 ms
Test accuracy of KNN classifier:  0.968


### Task 6 (3 points) <br/>

Develop **predict_proba** method of the classifier. For each object this method returns probability that the object belongs to each of the classes. 

For each object $x$ probability for each class is defined as:

$$
p_{c}(x) = \frac{g_{c}(x)}{\sum_{i=1}^{C} g_{i}(x)}
$$

where $C$ is number of classes.

Then, the object has a vector of probabilities:

$$
p(x) = (p_{1}(x), p_{2}(x), ..., p_{C}(x))
$$

Use neighbors weights as in Task 5.

In [46]:
# Create a class object
knn = KNNClassifier(max_dist=0.5, use_kd_tree=True, use_weights=True)

# Train the classifier
knn.fit(X_train, y_train)

# Make prediction using the trained classifier
%time y_test_predict_proba = knn.predict_proba(X_test) # measure time for prediction

# Example of the output
y_test_predict_proba[:10, :] # the first 10 rows



### Check your solution
ans = np.array([[0.046, 0.954],
                [0.962, 0.038]])
assert np.array_equal(np.round(y_test_predict_proba[:2], 3), ans), ('Check your solution.')

CPU times: user 923 ms, sys: 7.47 ms, total: 931 ms
Wall time: 964 ms
